In [ ]:
#original code: https://www.kaggle.com/code/nakanishiwataru/lb0-066-gnn-bert-hvec-concat-nn?scriptVersionId=260496553

# deps:
"""
https://www.kaggle.com/datasets/adamlogman/modred-dataset
https://www.kaggle.com/datasets/negoto/mordred-1-2-0-py3-none-any
https://www.kaggle.com/datasets/fridaycode/tg-smiles-pid-polyinfo-class
https://www.kaggle.com/datasets/henryjavier/torch-geometric-2-6-1
"""

In [ ]:
#Test offline install torch-molecule
!pip install /kaggle/input/torch-molecule-whl/torch_molecule-0.1.3-py3-none-any.whl --no-deps --no-index --find-links=file:///kaggle/input/torch-molecule-pkg

!pip install /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl

!pip install /kaggle/input/torch-geometric-2-6-1/torch_geometric-2.6.1-py3-none-any.whl

Looking in links: file:///kaggle/input/torch-molecule-pkg
Processing /kaggle/input/torch-molecule-whl/torch_molecule-0.1.3-py3-none-any.whl
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/kaggle/input/torch-molecule-whl/torch_molecule-0.1.3-py3-none-any.whl'

Processing /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl
Processing /kaggle/input/torch-geometric-2-6-1/torch_geometric-2.6.1-py3-none-any.whl


In [2]:
!pip install mordred --no-index --find-links=file:///kaggle/input/mordred-1-2-0-py3-none-any/

Looking in links: file:///kaggle/input/mordred-1-2-0-py3-none-any/
Processing /kaggle/input/mordred-1-2-0-py3-none-any/mordred-1.2.0-py3-none-any.whl
Processing /kaggle/input/mordred-1-2-0-py3-none-any/networkx-2.8.8-py3-none-any.whl (from mordred)
  Attempting uninstall: networkx
    Found existing installation: networkx 3.5
    Uninstalling networkx-3.5:
      Successfully uninstalled networkx-3.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.25.2 requires networkx>=3.0, but you have networkx 2.8.8 which is incompatible.
nx-cugraph-cu12 25.2.0 requires networkx>=3.2, but you have networkx 2.8.8 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu

In [4]:
%%writefile gnn_infer.py
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors, AllChem, Fragments, Lipinski
from rdkit.Chem import rdmolops
# Data paths
BASE_PATH = '/kaggle/input/neurips-open-polymer-prediction-2025/'
RDKIT_AVAILABLE = True
TARGETS = ['Tg', 'FFV', 'Tc', 'Density', 'Rg']
def get_canonical_smiles(smiles):
        """Convert SMILES to canonical form for consistency"""
        if not RDKIT_AVAILABLE:
            return smiles
        try:
            mol = Chem.MolFromSmiles(smiles)
            if mol:
                return Chem.MolToSmiles(mol, canonical=True)
        except:
            pass
        return smiles

#Cell 3: Robust Data Loading with Complete R-Group Filtering
"""
Load competition data with complete filtering of problematic polymer notation
"""

print("Loading competition data...")
train = pd.read_csv(BASE_PATH + 'train.csv')
test = pd.read_csv(BASE_PATH + 'test.csv')

print(f"   Training samples: {len(train)}")
print(f"   Test samples: {len(test)}")

def clean_and_validate_smiles(smiles):
    """Completely clean and validate SMILES, removing all problematic patterns"""
    if not isinstance(smiles, str) or len(smiles) == 0:
        return None
    
    # List of all problematic patterns we've seen
    bad_patterns = [
        '[R]', '[R1]', '[R2]', '[R3]', '[R4]', '[R5]', 
        "[R']", '[R"]', 'R1', 'R2', 'R3', 'R4', 'R5',
        # Additional patterns that cause issues
        '([R])', '([R1])', '([R2])', 
    ]
    
    # Check for any bad patterns
    for pattern in bad_patterns:
        if pattern in smiles:
            return None
    
    # Additional check: if it contains ] followed by [ without valid atoms, likely polymer notation
    if '][' in smiles and any(x in smiles for x in ['[R', 'R]']):
        return None
    
    # Try to parse with RDKit if available
    if RDKIT_AVAILABLE:
        try:
            mol = Chem.MolFromSmiles(smiles)
            if mol is not None:
                return Chem.MolToSmiles(mol, canonical=True)
            else:
                return None
        except:
            return None
    
    # If RDKit not available, return cleaned SMILES
    return smiles

# Clean and validate all SMILES
print("Cleaning and validating SMILES...")
train['SMILES'] = train['SMILES'].apply(clean_and_validate_smiles)
test['SMILES'] = test['SMILES'].apply(clean_and_validate_smiles)

# Remove invalid SMILES
invalid_train = train['SMILES'].isnull().sum()
invalid_test = test['SMILES'].isnull().sum()
train = train[train['SMILES'].notnull()].reset_index(drop=True)
test = test[test['SMILES'].notnull()].reset_index(drop=True)

print(f"   Final training samples: {len(train)}")
print(f"   Final test samples: {len(test)}")

def add_extra_data_clean(df_train, df_extra, target):
    """Add external data with thorough SMILES cleaning"""
    n_samples_before = len(df_train[df_train[target].notnull()])
    
    print(f"      Processing {len(df_extra)} {target} samples...")
    
    # Clean external SMILES
    df_extra['SMILES'] = df_extra['SMILES'].apply(clean_and_validate_smiles)
    
    # Remove invalid SMILES and missing targets
    before_filter = len(df_extra)
    df_extra = df_extra[df_extra['SMILES'].notnull()]
    df_extra = df_extra.dropna(subset=[target])
    after_filter = len(df_extra)
    
    print(f"      Kept {after_filter}/{before_filter} valid samples")
    
    if len(df_extra) == 0:
        print(f"      No valid data remaining for {target}")
        return df_train
    
    # Group by canonical SMILES and average duplicates
    df_extra = df_extra.groupby('SMILES', as_index=False)[target].mean()
    
    cross_smiles = set(df_extra['SMILES']) & set(df_train['SMILES'])
    unique_smiles_extra = set(df_extra['SMILES']) - set(df_train['SMILES'])

    # Fill missing values
    filled_count = 0
    for smile in df_train[df_train[target].isnull()]['SMILES'].tolist():
        if smile in cross_smiles:
            df_train.loc[df_train['SMILES']==smile, target] = \
                df_extra[df_extra['SMILES']==smile][target].values[0]
            filled_count += 1
    
    # Add unique SMILES
    extra_to_add = df_extra[df_extra['SMILES'].isin(unique_smiles_extra)].copy()
    if len(extra_to_add) > 0:
        for col in TARGETS:
            if col not in extra_to_add.columns:
                extra_to_add[col] = np.nan
        
        extra_to_add = extra_to_add[['SMILES'] + TARGETS]
        df_train = pd.concat([df_train, extra_to_add], axis=0, ignore_index=True)

    n_samples_after = len(df_train[df_train[target].notnull()])
    print(f'      {target}: +{n_samples_after-n_samples_before} samples, +{len(unique_smiles_extra)} unique SMILES')
    return df_train

# Load external datasets with robust error handling
print("\n📂 Loading external datasets...")

external_datasets = []

# Function to safely load datasets
def safe_load_dataset(path, target, processor_func, description):
    try:
        if path.endswith('.xlsx'):
            data = pd.read_excel(path)
        else:
            data = pd.read_csv(path)
        
        data = processor_func(data)
        external_datasets.append((target, data))
        print(f"   ✅ {description}: {len(data)} samples")
        return True
    except Exception as e:
        print(f"   ⚠️ {description} failed: {str(e)[:100]}")
        return False

# Load each dataset
safe_load_dataset(
    '/kaggle/input/tc-smiles/Tc_SMILES.csv',
    'Tc',
    lambda df: df.rename(columns={'TC_mean': 'Tc'}),
    'Tc data'
)

safe_load_dataset(
    '/kaggle/input/tg-smiles-pid-polyinfo-class/Tg_SMILES_class_pid_polyinfo_median.csv',
    'Tg', 
    lambda df: df[['SMILES', 'Tg']] if 'Tg' in df.columns else df,
    'TgSS enriched data'
)

safe_load_dataset(
    '/kaggle/input/smiles-extra-data/JCIM_sup_bigsmiles.csv',
    'Tg',
    lambda df: df[['SMILES', 'Tg (C)']].rename(columns={'Tg (C)': 'Tg'}),
    'JCIM Tg data'
)

safe_load_dataset(
    '/kaggle/input/smiles-extra-data/data_tg3.xlsx',
    'Tg',
    lambda df: df.rename(columns={'Tg [K]': 'Tg'}).assign(Tg=lambda x: x['Tg'] - 273.15),
    'Xlsx Tg data'
)

safe_load_dataset(
    '/kaggle/input/smiles-extra-data/data_dnst1.xlsx',
    'Density',
    lambda df: df.rename(columns={'density(g/cm3)': 'Density'})[['SMILES', 'Density']]
                .query('SMILES.notnull() and Density.notnull() and Density != "nylon"')
                .assign(Density=lambda x: x['Density'].astype(float) - 0.118),
    'Density data'
)

safe_load_dataset(
    '/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset4.csv',
    'FFV', 
    lambda df: df[['SMILES', 'FFV']] if 'FFV' in df.columns else df,
    'dataset 4'
)

# Integrate external data
print("\nIntegrating external data...")
train_extended = train[['SMILES'] + TARGETS].copy()

for target, dataset in external_datasets:
    print(f"   Processing {target} data...")
    train_extended = add_extra_data_clean(train_extended, dataset, target)

print(f"\n Final training data:")
print(f"   Original samples: {len(train)}")
print(f"   Extended samples: {len(train_extended)}")
print(f"   Gain: +{len(train_extended) - len(train)} samples")

for target in TARGETS:
    count = train_extended[target].notna().sum()
    original_count = train[target].notna().sum() if target in train.columns else 0
    gain = count - original_count
    print(f"   {target}: {count:,} samples (+{gain})")

print(f"\nData integration complete with clean SMILES!")

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, GATConv, global_mean_pool, global_max_pool
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from rdkit import Chem
from rdkit.Chem import Descriptors
import warnings
warnings.filterwarnings('ignore')

# データ読み込み
train = train_extended.copy()#pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train.csv')
test = pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/test.csv')

target_cols = ['Tg','FFV','Tc','Density','Rg']

def smiles_to_graph(smiles):
    """SMILES文字列を分子グラフに変換"""
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None

        # 原子特徴量の取得
        atom_features = []
        for atom in mol.GetAtoms():
            features = [
                atom.GetAtomicNum(),
                atom.GetDegree(),
                atom.GetFormalCharge(),
                int(atom.GetHybridization()),
                int(atom.GetIsAromatic()),
                atom.GetTotalNumHs(),
                int(atom.IsInRing()),
                atom.GetMass()
            ]
            atom_features.append(features)
        
        if len(atom_features) == 0:
            return None
            
        # エッジ（結合）情報の取得
        edge_indices = []
        edge_features = []
        
        for bond in mol.GetBonds():
            i = bond.GetBeginAtomIdx()
            j = bond.GetEndAtomIdx()
            
            bond_features = [
                bond.GetBondTypeAsDouble(),
                int(bond.IsInRing()),
                int(bond.GetIsConjugated()),
                int(bond.GetIsAromatic())
            ]
            
            # 無向グラフとして扱う
            edge_indices.extend([[i, j], [j, i]])
            edge_features.extend([bond_features, bond_features])
        
        # PyTorch Geometricのデータ形式に変換
        if len(edge_indices) == 0:
            # 結合がない場合（単原子分子）
            edge_index = torch.zeros((2, 0), dtype=torch.long)
            edge_attr = torch.zeros((0, 4), dtype=torch.float)
        else:
            edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()
            edge_attr = torch.tensor(edge_features, dtype=torch.float)
        
        x = torch.tensor(atom_features, dtype=torch.float)
        
        # 分子レベルの特徴量も追加
        mol_features = [
            Descriptors.MolWt(mol),
            Descriptors.NumHDonors(mol),
            Descriptors.NumHAcceptors(mol),
            Descriptors.TPSA(mol),
            Descriptors.NumRotatableBonds(mol),
            len(smiles)  # SMILES長
        ]
        
        data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, 
                   mol_features=torch.tensor(mol_features, dtype=torch.float))
        
        return data
    except:
        return None

class PolymerGNN(nn.Module):
    def __init__(self, atom_features_dim=8, edge_features_dim=4, mol_features_dim=6, 
                 hidden_dim=128, num_layers=2, num_targets=5, dropout=0.0):
        super(PolymerGNN, self).__init__()
        
        self.num_targets = num_targets
        self.dropout = dropout
        
        # Graph convolution layers
        self.convs = nn.ModuleList()
        self.batch_norms = nn.ModuleList()
        
        # 最初の層
        self.convs.append(GCNConv(atom_features_dim, hidden_dim))
        self.batch_norms.append(nn.BatchNorm1d(hidden_dim))
        
        # 中間層
        for _ in range(num_layers - 1):
            self.convs.append(GCNConv(hidden_dim, hidden_dim))
            self.batch_norms.append(nn.BatchNorm1d(hidden_dim))
        
        # 注意機構を追加
        self.attention = GATConv(hidden_dim, hidden_dim//4, heads=4, dropout=dropout)
        self.attention_bn = nn.BatchNorm1d(hidden_dim)
        
        # 分子レベル特徴量の処理
        self.mol_fc = nn.Sequential(
            nn.Linear(mol_features_dim, hidden_dim//2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim//2, hidden_dim//2)
        )
        
        # 最終予測層（各ターゲット別）
        combined_dim = hidden_dim * 2 + hidden_dim//2  # mean + max pooling + mol features
        
        self.predictors = nn.ModuleList()
        for _ in range(num_targets):
            predictor = nn.Sequential(
                nn.Linear(combined_dim, hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(hidden_dim, hidden_dim//2),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(hidden_dim//2, 1)
            )
            self.predictors.append(predictor)
    
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        mol_features = torch.stack([d.mol_features for d in data.to_data_list()]).to(x.device)
        
        # Graph convolutions
        for conv, bn in zip(self.convs, self.batch_norms):
            x = conv(x, edge_index)
            x = bn(x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
    
        # Attention layer
        x = self.attention(x, edge_index)
        x = self.attention_bn(x)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
    
        # Global pooling
        graph_mean = global_mean_pool(x, batch)
        graph_max = global_max_pool(x, batch)
    
        # mol features
        mol_emb = self.mol_fc(mol_features)
    
        # 結合
        graph_repr = torch.cat([graph_mean, graph_max, mol_emb], dim=1)
    
        # 予測
        predictions = [predictor(graph_repr) for predictor in self.predictors]
        return torch.cat(predictions, dim=1)

def create_data_loaders(df, target_cols, batch_size=32, is_train=True):
    """データローダーを作成"""
    graphs = []
    targets = []
    valid_indices = []
    
    for idx, row in df.iterrows():
        graph = smiles_to_graph(row['SMILES'])
        if graph is not None:
            graphs.append(graph)
            if is_train:
                # 単一ターゲットの場合は1次元、複数ターゲットの場合は2次元
                if len(target_cols) == 1:
                    target_value = row[target_cols[0]] if pd.notna(row[target_cols[0]]) else 0.0
                    targets.append(target_value)
                else:
                    target_values = [row[col] if pd.notna(row[col]) else 0.0 for col in target_cols]
                    targets.append(target_values)
            valid_indices.append(len(graphs) - 1)
    
    if is_train and len(targets) > 0:
        if len(target_cols) == 1:
            # 単一ターゲットの場合は1次元テンソル
            targets_tensor = torch.tensor(targets, dtype=torch.float)
            for i, graph in enumerate(graphs):
                graph.y = targets_tensor[i:i+1]  # 1次元を保持
        else:
            # 複数ターゲットの場合は2次元テンソル
            targets_tensor = torch.tensor(targets, dtype=torch.float)
            for i, graph in enumerate(graphs):
                graph.y = targets_tensor[i]
    
    if len(graphs) == 0:
        return None, valid_indices
    
    loader = DataLoader(graphs, batch_size=batch_size, shuffle=is_train)
    return loader, valid_indices

# wMAE計算のための重み計算
def calculate_wmae_weights(train_data, target_cols):
    """wMAE用の重みを計算"""
    weights = {}
    
    for target in target_cols:
        nt = train_data[target].notna().sum()
        train_values = train_data[target].dropna()
        rt = train_values.max() - train_values.min()
        wt = 1.0 / (np.sqrt(nt) * rt)
        weights[target] = wt
        print(f'{target}: nt={nt}, rt={rt:.4f}, wt={wt:.6f}')
    
    return weights

def calculate_wmae(y_true_dict, y_pred_dict, weights):
    """重み付き平均絶対誤差を計算"""
    total_wmae = 0.0
    total_weight = 0.0
    
    for target in y_true_dict.keys():
        if target in weights:
            mask = ~np.isnan(y_true_dict[target])
            if mask.sum() > 0:
                mae = np.mean(np.abs(y_true_dict[target][mask] - y_pred_dict[target][mask]))
                weighted_mae = weights[target] * mae
                total_wmae += weighted_mae
                total_weight += weights[target]
    
    return total_wmae / total_weight if total_weight > 0 else 0.0

# デバイス設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# wMAE重みを計算
wmae_weights = calculate_wmae_weights(train, target_cols)

# クロスバリデーション設定
kf = KFold(n_splits=5, shuffle=True, random_state=110)

# 結果保存用
test_preds = pd.DataFrame({'id': test['id']})
oof_preds = pd.DataFrame(index=train.index, columns=target_cols)
target_metrics = {}

# 各ターゲット別に学習（マルチタスク学習も可能だが、個別に学習）
for target_idx, target in enumerate(target_cols):
    print(f'\n==> Training for target: {target}')
    
    # 対象ターゲットが存在する行のみを使用
    mask = train[target].notnull()
    train_subset = train.loc[mask].copy()
    
    fold_rmses = []
    fold_maes = []
    test_fold_preds = np.zeros(len(test))
    
    for fold, (tr_idx, vl_idx) in enumerate(kf.split(train_subset), 1):
        print(f'  Fold {fold}...')
        
        train_fold = train_subset.iloc[tr_idx]
        valid_fold = train_subset.iloc[vl_idx]
        
        # データローダー作成
        train_loader, train_indices = create_data_loaders(train_fold, [target], batch_size=32, is_train=True)
        valid_loader, valid_indices = create_data_loaders(valid_fold, [target], batch_size=32, is_train=True)
        test_loader, test_indices = create_data_loaders(test, target_cols, batch_size=32, is_train=False)
        
        if train_loader is None or valid_loader is None:
            print(f'    Skipping fold {fold} due to data issues')
            continue
        
        # モデル初期化
        model = PolymerGNN(num_targets=1).to(device)
        optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001, weight_decay=5e-4)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=10)
        criterion = nn.MSELoss()
        
        # 訓練データのターゲット値を正規化
        train_targets = []
        for batch in train_loader:
            # バッチ内の各グラフのターゲット値を取得
            batch_targets = batch.y
            if batch_targets.dim() == 1:
                train_targets.extend(batch_targets.tolist())
            else:
                train_targets.extend(batch_targets[:, 0].tolist())
        
        target_scaler = StandardScaler()
        train_targets_array = np.array(train_targets).reshape(-1, 1)
        target_scaler.fit(train_targets_array)
        
        # 学習ループ
        best_val_loss = float('inf')
        patience_counter = 0
        max_patience = 20
        
        for epoch in range(200):
            # 訓練
            model.train()
            train_loss = 0
            num_batches = 0
            
            for batch in train_loader:
                batch = batch.to(device)
                optimizer.zero_grad()
                
                # ターゲット値を正規化
                batch_targets = batch.y
                if batch_targets.dim() == 1:
                    targets_normalized = target_scaler.transform(batch_targets.cpu().numpy().reshape(-1, 1))
                else:
                    targets_normalized = target_scaler.transform(batch_targets[:, 0:1].cpu().numpy())
                targets_normalized = torch.tensor(targets_normalized, dtype=torch.float).to(device)
                
                out = model(batch)
                loss = criterion(out.squeeze(), targets_normalized.squeeze())
                loss.backward()
                optimizer.step()
                
                train_loss += loss.item()
                num_batches += 1
            
            # 検証
            model.eval()
            val_loss = 0
            val_preds = []
            val_targets = []
            
            with torch.no_grad():
                for batch in valid_loader:
                    batch = batch.to(device)
                    batch_targets = batch.y
                    if batch_targets.dim() == 1:
                        targets_normalized = target_scaler.transform(batch_targets.cpu().numpy().reshape(-1, 1))
                    else:
                        targets_normalized = target_scaler.transform(batch_targets[:, 0:1].cpu().numpy())
                    targets_normalized = torch.tensor(targets_normalized, dtype=torch.float).to(device)
                    
                    out = model(batch)
                    loss = criterion(out.squeeze(), targets_normalized.squeeze())
                    val_loss += loss.item()
                    
                    # 予測値を元のスケールに戻す
                    preds_original = target_scaler.inverse_transform(out.cpu().numpy().reshape(-1, 1))
                    val_preds.extend(preds_original.flatten())
                    
                    if batch_targets.dim() == 1:
                        val_targets.extend(batch_targets.cpu().numpy())
                    else:
                        val_targets.extend(batch_targets[:, 0].cpu().numpy())
            
            val_loss /= len(valid_loader)
            scheduler.step(val_loss)
            
            # Early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
                # ベストモデルを保存
                best_model_state = model.state_dict().copy()
                torch.save(best_model_state, f'NN_{target}_fold_{fold}_best.pth')
            else:
                patience_counter += 1
                if patience_counter >= max_patience:
                    break
        
        print("training epochs:", epoch)
        # ベストモデルをロード
        model.load_state_dict(best_model_state)
        
        # 最終評価
        model.eval()
        val_preds = []
        val_targets = []
        
        with torch.no_grad():
            for batch in valid_loader:
                batch = batch.to(device)
                out = model(batch)
                preds_original = target_scaler.inverse_transform(out.cpu().numpy().reshape(-1, 1))
                val_preds.extend(preds_original.flatten())
                
                batch_targets = batch.y
                if batch_targets.dim() == 1:
                    val_targets.extend(batch_targets.cpu().numpy())
                else:
                    val_targets.extend(batch_targets[:, 0].cpu().numpy())
        
        val_preds = np.array(val_preds)
        val_targets = np.array(val_targets)
        
        rmse = np.sqrt(mean_squared_error(val_targets, val_preds))
        mae = mean_absolute_error(val_targets, val_preds)
        
        fold_rmses.append(rmse)
        fold_maes.append(mae)
        print(f'    Fold {fold} RMSE: {rmse:.4f}, MAE: {mae:.4f}')
        
        # OOF予測を保存
        valid_original_indices = valid_fold.index[valid_indices]
        oof_preds.loc[valid_original_indices, target] = val_preds
        
        # テストデータの予測
        if test_loader is not None:
            test_preds_fold = []
            with torch.no_grad():
                for batch in test_loader:
                    batch = batch.to(device)
                    out = model(batch)
                    preds_original = target_scaler.inverse_transform(out.cpu().numpy().reshape(-1, 1))
                    test_preds_fold.extend(preds_original.flatten())
            
            test_fold_preds[test_indices] += np.array(test_preds_fold) / kf.n_splits
    
    if len(fold_rmses) > 0:
        test_preds[target] = test_fold_preds
        avg_rmse = np.mean(fold_rmses)
        avg_mae = np.mean(fold_maes)
        target_metrics[target] = {'rmse': avg_rmse, 'mae': avg_mae}
        
        print(f'  >>> Avg RMSE for {target}: {avg_rmse:.4f}')
        print(f'  >>> Avg MAE for {target}: {avg_mae:.4f}')

# 全体評価
print(f'\n=== Overall Evaluation ===')

# 全体RMSE計算
all_oof = []
all_true = []
for target in target_cols:
    mask = train[target].notnull() & oof_preds[target].notnull()
    if mask.sum() > 0:
        all_oof.extend(oof_preds.loc[mask, target].astype(float).values)
        all_true.extend(train.loc[mask, target].values)

if len(all_oof) > 0:
    overall_rmse = np.sqrt(mean_squared_error(all_true, all_oof))
    print(f'Overall OOF RMSE: {overall_rmse:.4f}')

# wMAE計算
y_true_dict = {}
y_pred_dict = {}

for target in target_cols:
    mask = train[target].notnull() & oof_preds[target].notnull()
    if mask.sum() > 0:
        y_true_dict[target] = train.loc[mask, target].values
        y_pred_dict[target] = oof_preds.loc[mask, target].astype(float).values

if y_true_dict:
    overall_wmae = calculate_wmae(y_true_dict, y_pred_dict, wmae_weights)
    print(f'Overall OOF wMAE: {overall_wmae:.6f}')

# 各targetの詳細メトリクス表示
print(f'\n=== Target-wise Metrics ===')
for target in target_cols:
    if target in target_metrics:
        metrics = target_metrics[target]
        weight = wmae_weights[target]
        weighted_mae = weight * metrics['mae']
        print(f'{target:8s}: RMSE={metrics["rmse"]:.4f}, MAE={metrics["mae"]:.4f}, '
              f'Weight={weight:.6f}, Weighted MAE={weighted_mae:.6f}')

# 提出ファイル作成
submission = test_preds[['id'] + target_cols]
submission.to_csv('gnn_sub.csv', index=False)
print(f'\n=== Submission Preview ===')
print(submission.head())

Writing gnn_infer.py


In [6]:
!python gnn_infer.py

Loading competition data...
   Training samples: 7973
   Test samples: 3
Cleaning and validating SMILES...
   Final training samples: 7973
   Final test samples: 3

📂 Loading external datasets...
   ✅ Tc data: 874 samples
   ✅ TgSS enriched data: 7208 samples
   ✅ JCIM Tg data: 662 samples
   ✅ Xlsx Tg data: 501 samples
   ✅ Density data: 786 samples
   ✅ dataset 4: 862 samples

Integrating external data...
   Processing Tc data...
      Processing 874 Tc samples...
      Kept 874/874 valid samples
      Tc: +129 samples, +129 unique SMILES
   Processing Tg data...
      Processing 7208 Tg samples...
      Kept 7208/7208 valid samples
      Tg: +7168 samples, +1923 unique SMILES
   Processing Tg data...
      Processing 662 Tg samples...
      Kept 662/662 valid samples
      Tg: +66 samples, +58 unique SMILES
   Processing Tg data...
      Processing 501 Tg samples...
      Kept 501/501 valid samples
      Tg: +499 samples, +499 unique SMILES
   Processing Density data...
      Proces